# Mapping Cybera Members

In [ ]:
import pandas as pd
#import geopandas
#import pgeocode
#nomi = pgeocode.Nominatim('ca')
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import folium
try:
    import haversine as hs
except:
    !pip install --user haversine
    import haversine as hs
try:
    import pynetbox
except:
    !pip install --user pynetbox==6.6.2
    import pynetbox
print('Libraries imported')

## Getting Member Locations

Using the [Netbox](https://github.com/netbox-community/pynetbox) API, which equires a token, this must be run from on the Cybera network.

In [ ]:
NETBOX_API_KEY = ''

try:
    nb = pynetbox.api('https://netbox.cybera.ca', token=NETBOX_API_KEY)
    sites = nb.dcim.sites
    locations = []
    for site in sites.all():
        locations.append([site.url.split('/')[6], site, site.latitude, site.longitude, site.physical_address])
    df = pd.DataFrame(locations, columns=['ID', 'Site', 'Latitude', 'Longitude', 'Address'])
    df.to_csv('data/cybera-members.csv', index=False)
except:
    df = pd.read_csv('data/cybera-members.csv')
display(df)

Drop any NaN values

In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)
df

Create a Folium map

In [ ]:
locations_map = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=5)
for i in range(0,len(df)):
    folium.Marker([df.iloc[i]['Latitude'], df.iloc[i]['Longitude']], popup=df.iloc[i]['Site']).add_to(locations_map)
locations_map

Which unique cities are there now?

In [ ]:
print(len(df['City'].unique()))
df['City'].unique()

Sort values by latitude and calculate the distance to the next location.

In [ ]:
df = df.sort_values(by='Latitude')
df.reset_index(inplace=True, drop=True)

distances_list = []
for i in range(len(df)):
    distances_list.append([df.iloc[i]['City'], df.iloc[i]['Site'], df.iloc[i-1]['Site'], 
                      hs.haversine((df.iloc[i]['Latitude'], df.iloc[i]['Longitude']), 
                                   (df.iloc[i-1]['Latitude'], df.iloc[i-1]['Longitude']))])
dbc = pd.DataFrame(distances_list, columns=['City', 'Site', 'Next Site', 'Distance'])
dbc

Which locations are close together?

In [ ]:
close_distance = 75
display(dbc[dbc['Distance'] < close_distance])

Replace smaller cities with nearby larger city

In [ ]:
cities_to_replace = {
    'Taber':'Lethbridge',
    'Dunmore':'Medicine Hat',
    'Morley':'Canmore',
    'Banff':'Canmore',
    'Ponoka':'Lacombe',
    'Wetaskiwin':'Camrose',
    'Leduc':'Edmonton',
    'Nisku':'Edmonton',
    'Spruce Grove':'Edmonton',
    'Stony Plain':'Edmonton',
    'St. Albert':'Edmonton',
    'Morinville':'Edmonton',
    'Elk Point':'Lac La Biche',
    'Grouard':'High Prairie',
    'Red Earth Creek':'High Prairie',
    'Grimshaw':'Peace River',
    'Innisfail':'Olds',
    'Three Hills':'Olds',
    'Frog Lake':'Bonnyville',
}

df['PixelName'] = df['City'].replace(cities_to_replace)
df

In [ ]:
print(len(df['PixelName'].unique()))
df['PixelName'].unique()

Make a new map with just the larger cities.

In [ ]:
new_map = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=5)
for city in df['PixelName'].unique():
    lat = df[df['City']==city]['Latitude'].mean()
    lon = df[df['City']==city]['Longitude'].mean()
    folium.Marker([lat, lon], popup=city).add_to(new_map)

original_map_cities = pd.read_csv('data/original_map_cities.csv')
for row in original_map_cities.iterrows():
    folium.CircleMarker(location=[row[1]['Latitude'], row[1]['Longitude']], popup=row[1]['Name'], radius=5, color='red').add_to(new_map)
new_map

Looks like Draton Valley and Slave Lake were missing in the original dataset. We'll have to fix that by adding two more pixels.

Create a pixels dataframe from south to north, since that's how the pixels are wired.

Also correct the latitude and longitude for Edmonton and Calgary, since we were just using the mean values including nearby cities.

In [ ]:
pixels = pd.DataFrame()
pixels['City'] = list(df['PixelName'].unique())
lats = []
lons = []
for i in range(len(pixels)):
    lats.append(df[df['PixelName']==pixels.iloc[i]['City']].iloc[0]['Latitude'])
    lons.append(df[df['PixelName']==pixels.iloc[i]['City']].iloc[0]['Longitude'])
pixels['Latitude'] = lats
pixels['Longitude'] = lons

pixels.loc[pixels['City']=='Edmonton', 'Latitude'] = 53.5444
pixels.loc[pixels['City']=='Edmonton', 'Longitude'] = -113.4909
pixels.loc[pixels['City']=='Calgary', 'Latitude'] = 51.0486
pixels.loc[pixels['City']=='Calgary', 'Longitude'] = -114.0708

pixels = pixels.sort_values('Latitude').reset_index(drop=True)
pixels['Pixel'] = pixels.index
pixels

Make a column in the df for which pixel represents that location.

In [ ]:
df2 = pd.merge(df, pixels, left_on='PixelName', right_on='City', how='left')
df2 = df2[df2.columns.drop(list(df2.filter(regex='_y')))]  # drop the _y columns
df2 = df2.rename(columns={'Latitude_x':'Latitude', 'Longitude_x':'Longitude', 'City_x': 'City'})
df2

Export `df2` to a new CSV file.

In [ ]:
df2.to_csv('data/cybera-members-collated.csv', index=False)

Add a column to the pixels dataframe with pixel colors representing how many Cybera members are represented by a pixel.

In [ ]:
counts = pd.DataFrame(df.groupby('PixelName')['PixelName'].count())
counts.columns = ['Count']  # rename the column
counts = counts.reset_index()  # flaten the dataframe
pixels = pd.merge(pixels, counts, left_on='City', right_on='PixelName').drop('PixelName', axis=1)  # merge pixels with counts
pixels

In [ ]:
def map_color(n, max_n=30):
    start_color = (249, 157, 42)
    end_color = (0, 168, 183)
    r = int(start_color[0] + (end_color[0] - start_color[0]) * n / max_n)
    g = int(start_color[1] + (end_color[1] - start_color[1]) * n / max_n)
    b = int(start_color[2] + (end_color[2] - start_color[2]) * n / max_n)
    return (r, g, b)

pixels['Color'] = pixels['Count'].apply(map_color)
pixels

Output a `status.csv` file for micropython to download.

In [ ]:
status = pd.DataFrame(pixels['Color'].tolist()).reset_index()
status.columns = columns=['LED', 'Red', 'Green', 'Blue']
status.to_csv('../docs/status.csv', index=False)
status

Display the colors

In [ ]:
'''
from IPython.display import HTML
for row in status.iterrows():
    color = tuple(row[1][['Red', 'Green', 'Blue']])
    html = f'<div style="width: 50px; height: 50px; background-color: rgb{color};"></div>'
    display(HTML(html))
'''

Create a map with the colors.

In [ ]:
colored_map = folium.Map(location=[pixels['Latitude'].mean(), pixels['Longitude'].mean()], zoom_start=5, tiles='Stamen Toner')
for row in pixels.iterrows():
    rgb_color = row[1]['Color']
    hex_color = '#%02x%02x%02x' % (rgb_color[0], rgb_color[1], rgb_color[2])
    folium.CircleMarker(location=[row[1]['Latitude'], row[1]['Longitude']], popup=str(row[1]['Pixel']) +' '+ row[1]['City'], radius=2, color=hex_color).add_to(colored_map)
colored_map

## Create a Map for CNC Toolpaths

In [ ]:
pixels.columns

In [ ]:
fig0 = go.Figure(data=go.Scattergeo(
    lat = pixels['Latitude'],
    lon = pixels['Longitude'],
    text = pixels['City']
)
               )

fig0.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        #landcolor = "rgb(212, 212, 212)",
        landcolor = "rgb(255, 255, 255)",
        subunitcolor = "rgb(0, 0, 0)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = False,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='Cybera Member locations',
    height=600,
    width=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig0.update_geos(lataxis_range=[48,61],
               lonaxis_range=[-120,-110])
print('Cybera Member Map')
#fig0.write_html('output/membermap.html')
fig0.show()

Export an SVG for import into the CAM program

In [ ]:
try:
    fig0.write_image('output/membermap.svg')
except:
    !pip install --user kaleido
    fig0.write_image('output/membermap.svg')

## Wiring Diagrams

In [ ]:
px.line(pixels, y='Latitude', x='Longitude', text='Pixel', hover_data=['Pixel','City','Latitude','Longitude'], height=1000, width=650).update_traces(mode='markers+lines+text', marker=dict(size=8), textposition='top center')

In [ ]:
fig1 = px.line(pixels, y='Latitude', x='Longitude', height=1000, width=650)
fig1.add_scatter(y=pixels['Latitude'], x=pixels['Longitude'], text=pixels['City'], mode='markers+text', textposition='middle right', marker=dict(size=10))
fig1.add_scatter(y=pixels['Latitude'], x=pixels['Longitude'], text=pixels.index, mode='markers+text', textposition='bottom left', marker=dict(size=10))
fig1.update_xaxes(showticklabels=False, title=None).update_yaxes(showticklabels=False, title=None)
fig1.update_layout(showlegend=False, plot_bgcolor='rgba(0,0,0,0)', margin=dict(l=0, r=0, t=0, b=0))
fig1.show()

### Reversed mode for labelling the back of the wood

In [ ]:
backwards_map = pixels.copy()
backwards_map['Longitude_reversed'] = backwards_map['Longitude'] * -1
fig2 = px.line(backwards_map, y='Latitude', x='Longitude_reversed', height=1030, width=650, title='Reversed Map For Wiring')
fig2.add_scatter(y=backwards_map['Latitude'], x=backwards_map['Longitude_reversed'], text=backwards_map['City'], mode='markers+text', textposition='middle right', marker=dict(size=10))
fig2.add_scatter(y=backwards_map['Latitude'], x=backwards_map['Longitude_reversed'], text=backwards_map.index, mode='markers+text', textposition='bottom left', marker=dict(size=10))
fig2.update_xaxes(showticklabels=False, title=None).update_yaxes(showticklabels=False, title=None)
fig2.update_layout(showlegend=False, plot_bgcolor='rgba(0,0,0,0)', margin=dict(l=0, r=0, t=30, b=0))
#fig2.write_image('output/reversed_wire_map.png')
fig2.show()

## Enable Dark Mode

In [ ]:
import pandas as pd
status = pd.read_csv('../docs/status.csv')
for n in range(len(status)):
    status.iloc[n][1:] = 0
status.to_csv('../docs/status.csv', index=False)
status